### **Input**

In [2]:
input_sentence = input("Enter input sentence")

Enter input sentence मुझे खाना बहुत पसंद है


### **Preprocessing Data**

In [3]:
stop_words  = open("stopwords.txt", "r")

In [4]:
stop_words_list = []
for line in stop_words:
    line = line.strip('\n').split()
    stop_words_list.append(line[0])

In [5]:
import inltk
from inltk.inltk import tokenize, get_embedding_vectors, predict_next_words, get_sentence_encoding, get_sentence_similarity, get_similar_sentences

In [6]:
def preprocess_sentence(sentence):
    tokenized_ips = tokenize(sentence, 'hi')
    for i in range(len(tokenized_ips)):
        tokenized_ips[i] = tokenized_ips[i][1:]
    preprocessed_ips = ""
    for token in tokenized_ips:
        if token not in stop_words_list:
            preprocessed_ips += (" "+token)
    return preprocessed_ips

In [7]:
preprocess_sentence(input_sentence)

' मुझे खाना पसंद'

### **Ambiguous Word Input**

In [8]:
amb_word = input("Enter ambiguous word:")

Enter ambiguous word: खाना


In [9]:
import pandas as pd
from googletrans import Translator

import nltk
import string
import re

from nltk.stem import WordNetLemmatizer

from englisttohindi.englisttohindi import EngtoHindi

In [10]:
def HindiLemmetizer(word):
    temp=translator.translate(word).text
    updated_temp=lemmatizer.lemmatize(temp)
    return (EngtoHindi(updated_temp)).convert

### **pyiwn**

#### Data Gathering

In [11]:
import pyiwn

In [12]:
iwn = pyiwn.IndoWordNet()

2023-05-16:08:12:27,23 INFO     [iwn.py:43] Loading hindi language synsets...


In [13]:
synsets = iwn.synsets(amb_word)

In [14]:
synsets

[Synset('खाद्य वस्तु.noun.20'),
 Synset('भोजन.noun.1830'),
 Synset('ख़ाना.noun.2896'),
 Synset('दराज़.noun.4652'),
 Synset('खाना.noun.4653'),
 Synset('खाना.verb.4655'),
 Synset('भक्षण.noun.7467'),
 Synset('श्रेणी.noun.8099'),
 Synset('हड़पना.verb.9339'),
 Synset('खाना.verb.10621'),
 Synset('चाटना.verb.12173'),
 Synset('खाना.verb.12174'),
 Synset('पीना.verb.12175'),
 Synset('खाना.verb.12176'),
 Synset('खाना.verb.12177'),
 Synset('खाना.verb.12194'),
 Synset('खाना.verb.13868'),
 Synset('खाना.verb.36034'),
 Synset('खाना.verb.37179'),
 Synset('भोजन.noun.38625'),
 Synset('खाना.verb.39260'),
 Synset('ख़ाना.noun.40160')]

In [15]:
for synset in synsets:
    print(synset.lemma_names())

['खाद्य वस्तु', 'खाद्य पदार्थ', 'खाद्यवस्तु', 'खाद्यपदार्थ', 'आहार', 'खाद्य', 'भोज्य पदार्थ', 'खाद्य सामग्री', 'खाद्य-सामग्री', 'आहार पदार्थ', 'अन्न', 'आहर', 'खाना', 'फूड', 'रसद', 'अर्क', 'इरा', 'तआम']
['भोजन', 'खाना', 'आहार', 'अन्न', 'रसोई', 'अशन', 'असन', 'आहर', 'रोटी', 'डाइट', 'ज्योनार', 'जेवन']
['ख़ाना', 'खाना', 'कोठा', 'बॉक्स', 'बाक्स']
['दराज़', 'दराज', 'ख़ाना', 'खाना']
['खाना', 'ख़ाना', 'घर', 'कोठा', 'गोटी घर']
['खाना', 'अहारना', 'मुँह चलाना', 'मुँह लगाना']
['भक्षण', 'खाना', 'अभ्याहार', 'चरण']
['श्रेणी', 'दर्जा', 'वर्ग', 'कोटि', 'समूह', 'गुट', 'कैटिगरी', 'कटेगरी', 'तबका', 'तबक़ा', 'ख़ाना', 'खाना']
['हड़पना', 'कब्ज़ा करना', 'हथियाना', 'गटकना', 'पचाना', 'पचा लेना', 'हज़म करना', 'हजम करना', 'हज्म करना', 'खाना', 'घोंटना', 'डकारना', 'कब्ज़ा जमाना', 'कब्ज़ा जमा लेना', 'मारना', 'ढाँपना']
['खाना']
['चाटना', 'खाना', 'दिमाग चाटना', 'दिमाग खाना', 'दिमाग खाली करना', 'दिमाग परेशान करना', 'दिमाग़ चाटना', 'दिमाग़ खाना', 'दिमाग़ खाली करना', 'दिमाग़ परेशान करना', 'दिमाग पिघलाना', 'दिमाग़ पिघलाना'

In [16]:
synonyms_set = set()
for synset in synsets: 
    add = False
    for lst in synset.lemma_names():
        if lst not in synonyms_set and lst != amb_word:
            synonyms_set.add(lst)
            break
synonyms_set

{'अहारना',
 'आहार',
 'ख़ाना',
 'खाद्य वस्तु',
 'घर',
 'चाटना',
 'जीमना',
 'दराज़',
 'पीना',
 'बरबाद करना',
 'बर्बाद करना',
 'भक्षण',
 'भोजन',
 'रिश्वत लेना',
 'श्रेणी',
 'हड़पना'}

### **Cosine Similarity**

In [10]:
from inltk.inltk import tokenize, get_embedding_vectors, predict_next_words, get_sentence_encoding, get_sentence_similarity, get_similar_sentences

In [17]:
sent_list = []
for syn in synonyms_set:
    dummy = input_sentence
    sent_list.append(dummy.replace(amb_word, syn))
print(sent_list)

['मुझे दराज़ बहुत पसंद है', 'मुझे खाद्य वस्तु बहुत पसंद है', 'मुझे भक्षण बहुत पसंद है', 'मुझे आहार बहुत पसंद है', 'मुझे पीना बहुत पसंद है', 'मुझे घर बहुत पसंद है', 'मुझे बर्बाद करना बहुत पसंद है', 'मुझे बरबाद करना बहुत पसंद है', 'मुझे चाटना बहुत पसंद है', 'मुझे जीमना बहुत पसंद है', 'मुझे अहारना बहुत पसंद है', 'मुझे भोजन बहुत पसंद है', 'मुझे हड़पना बहुत पसंद है', 'मुझे श्रेणी बहुत पसंद है', 'मुझे रिश्वत लेना बहुत पसंद है', 'मुझे ख़ाना बहुत पसंद है']


In [18]:
high_sim_sent = []
for sent in sent_list:
    sim = get_sentence_similarity(sent, input_sentence, 'hi')
    print(sent, sim)
    if sim >= 0.9:
        high_sim_sent.append(sent)
print(":")
print(high_sim_sent)

मुझे दराज़ बहुत पसंद है 0.8244850039482117


मुझे खाद्य वस्तु बहुत पसंद है 0.8151918649673462


मुझे भक्षण बहुत पसंद है 0.865024745464325


मुझे आहार बहुत पसंद है 0.9283260703086853


मुझे पीना बहुत पसंद है 0.8965064287185669


...

मुझे भोजन बहुत पसंद है 0.9286918640136719


मुझे हड़पना बहुत पसंद है 0.896979570388794


मुझे श्रेणी बहुत पसंद है 0.7833545804023743


मुझे रिश्वत लेना बहुत पसंद है 0.8873673677444458


मुझे ख़ाना बहुत पसंद है 0.8660592436790466
:
['मुझे आहार बहुत पसंद है', 'मुझे बर्बाद करना बहुत पसंद है', 'मुझे भोजन बहुत पसंद है']


In [19]:
ppc_high_sim_sent = []
for sent in high_sim_sent:
    ppc_high_sim_sent.append(preprocess_sentence(sent))

In [20]:
ppc_high_sim_sent

[' मुझे आहार पसंद', ' मुझे बर्बाद पसंद', ' मुझे भोजन पसंद']

### **MuRIL**

#### Model

In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

In [22]:
def get_model(model_url, max_seq_length):
    inputs = dict(
        input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
        input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
        input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
    )

    muril_layer = hub.KerasLayer(model_url, trainable=True)
    outputs = muril_layer(inputs)

    assert 'sequence_output' in outputs
    assert 'pooled_output' in outputs
    assert 'encoder_outputs' in outputs
    assert 'default' in outputs
    return tf.keras.Model(inputs=inputs,outputs=outputs["pooled_output"]), muril_layer

In [23]:
def create_input(input_strings, tokenizer, max_seq_length):
    input_ids_all, input_mask_all, input_type_ids_all = [], [], []
    for input_string in input_strings:
        input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
        input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        sequence_length = min(len(input_ids), max_seq_length)
    
        if len(input_ids) >= max_seq_length:
            input_ids = input_ids[:max_seq_length]
        else:
            input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

        input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

        input_ids_all.append(input_ids)
        input_mask_all.append(input_mask)
        input_type_ids_all.append([0] * max_seq_length)
  
    return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

In [24]:
def encode(input_text):
    input_ids, input_mask, input_type_ids = create_input(input_text, 
                                                       tokenizer, 
                                                       max_seq_length)
    inputs = dict(
      input_word_ids=input_ids,
      input_mask=input_mask,
      input_type_ids=input_type_ids,
    )
    return muril_model(inputs)

In [25]:
max_seq_length = 128
muril_model, muril_layer = get_model(model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=max_seq_length)
     

vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

2023-05-16:08:13:51,15 INFO     [resolver.py:109] Using C:\Users\thakr\AppData\Local\Temp\tfhub_modules to cache modules.


#### Returning best Euclidean distance sentence

In [26]:
sentences = []
for sentence in ppc_high_sim_sent:
    s = sentence.strip()
    sentences.append(s.split(" "))
sentences

[['मुझे', 'आहार', 'पसंद'],
 ['मुझे', 'बर्बाद', 'पसंद'],
 ['मुझे', 'भोजन', 'पसंद']]

In [35]:
def generate_muril_output(sent):
    embeddings = encode(sent)
    tot_dist = 0
    for index in range(len(sent)):
        tot_dist += distance.euclidean(np.array(embeddings[index]), np.array(embeddings[(index+1)%len(sent)]))
        print(sent[index], "\t", sent[(index+1)%len(sent)], "\t:", distance.euclidean(np.array(embeddings[index]), np.array(embeddings[(index+1)%len(sent)])))
    print("avg : ", tot_dist/len(sent), end = "\n\n")
    return tot_dist/len(sent)

In [36]:
for sentence in sentences:
    print("->", sentence[1])
    generate_muril_output(sentence)

-> आहार
मुझे 	 आहार 	: 0.013818785548210144
आहार 	 पसंद 	: 0.011560117825865746
पसंद 	 मुझे 	: 0.012692625634372234
avg :  0.012690509669482708

-> बर्बाद
मुझे 	 बर्बाद 	: 0.011643392965197563
बर्बाद 	 पसंद 	: 0.008085211738944054
पसंद 	 मुझे 	: 0.012692625634372234
avg :  0.010807076779504618

-> भोजन
मुझे 	 भोजन 	: 0.012454302981495857
भोजन 	 पसंद 	: 0.010638106614351273
पसंद 	 मुझे 	: 0.012692625634372234
avg :  0.011928345076739788

